# Trabalho 2

In [3043]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [3044]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 768
largura = 1366
qtd_texturas = 50
window = glfw.create_window(largura, altura, "Trabalho 2", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [3045]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [3046]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [3047]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [3048]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [3049]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [3050]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [3051]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [3052]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [3053]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model



In [3054]:
glEnable(GL_TEXTURE_2D)
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [3055]:
vertices_list = []    
textures_coord_list = []

### Vamos carregar cada modelo e definir funções para desenhá-los

In [3056]:
modelo = load_model_from_file('terreno/pedra.obj')

print('Processando modelo piso.obj. Vertice inicial:',len(vertices_list))
ini_pedra = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo piso.obj. Vertice final:',len(vertices_list))
fim_pedra = len(vertices_list)

load_texture_from_file(1,'terreno/piso-2.jpg')


Processando modelo piso.obj. Vertice inicial: 0
Processando modelo piso.obj. Vertice final: 6


In [3057]:
modelo = load_model_from_file('terreno/grama.obj')

print('Processando modelo grama.obj. Vertice inicial:',len(vertices_list))
ini_grama = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo grama.obj. Vertice final:',len(vertices_list))
fim_grama = len(vertices_list)

load_texture_from_file(4,'terreno/grama-1.jpg')


Processando modelo grama.obj. Vertice inicial: 6
Processando modelo grama.obj. Vertice final: 12


In [3058]:
modelo = load_model_from_file('casa/casa.obj')

print('Processando modelo casa.obj. Vertice inicial:',len(vertices_list))
ini_casa = len(vertices_list)
vertices_text = []
faces_visited = []
text_id = [2,5,6,7,8,9,10,11,12]
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], 'vertice inicial: ', len(vertices_list))
        faces_visited.append(face[2])
        vertices_text.append(len(vertices_list))
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo casa.obj. Vertice final:',len(vertices_list))
fim_casa = len(vertices_list)
    
vertices_text.append(fim_casa)

load_texture_from_file(2,'casa/metal_dark_brown.jpg')
load_texture_from_file(5,'casa/metal_grey.jpg')
load_texture_from_file(6,'casa/wood_brown.jpg')
load_texture_from_file(7,'casa/plaster_light_brown.jpg')
load_texture_from_file(8,'casa/plaster_sand.jpg')
load_texture_from_file(9,'casa/plate_grey.jpg')
load_texture_from_file(10,'casa/wood_balls_brown.jpg')
load_texture_from_file(11,'casa/foundation_brown_brick.jpg')
load_texture_from_file(12,'casa/glass_window.jpg')


Processando modelo casa.obj. Vertice inicial: 12
metal_dark_brown vertice inicial:  12
metal_grey vertice inicial:  469578
wood_brown vertice inicial:  571176
plaster_light_brown vertice inicial:  580644
plaster_sand vertice inicial:  582819
plate_grey vertice inicial:  583779
wood_balls_brown vertice inicial:  583809
foundation_brown_brick vertice inicial:  584145
glass_window vertice inicial:  585381
Processando modelo casa.obj. Vertice final: 585477


In [3059]:
modelo = load_model_from_file('sofa/sofa.obj')

print('Processando modelo sofa.obj. Vertice inicial:',len(vertices_list))
ini_sofa = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo sofa.obj. Vertice final:',len(vertices_list))
fim_sofa = len(vertices_list)

load_texture_from_file(13,'sofa/sofa.jpg')

Processando modelo sofa.obj. Vertice inicial: 585477
Processando modelo sofa.obj. Vertice final: 590289


In [3060]:
modelo = load_model_from_file('tv/tv.obj')

print('Processando modelo tv.obj. Vertice inicial:',len(vertices_list))
ini_tv = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo tv.obj. Vertice final:',len(vertices_list))
fim_tv = len(vertices_list)

load_texture_from_file(14,'tv/tv.png')

Processando modelo tv.obj. Vertice inicial: 590289
Processando modelo tv.obj. Vertice final: 1263057


In [3061]:
modelo = load_model_from_file('arvore/arvore.obj')

print('Processando modelo arvore.obj. Vertice inicial:',len(vertices_list))
ini_arvore = len(vertices_list)
arvore_vertices_text = []
faces_visited = []
arvore_text_id = [15,16]
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], 'vertice inicial: ', len(vertices_list))
        faces_visited.append(face[2])
        arvore_vertices_text.append(len(vertices_list))
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo arvore.obj. Vertice final:',len(vertices_list))
fim_arvore = len(vertices_list)
    
arvore_vertices_text.append(fim_arvore)

load_texture_from_file(15,'arvore/tronco.jpg')
load_texture_from_file(16,'arvore/folha.jpg')


Processando modelo arvore.obj. Vertice inicial: 1263057
Materiais.001 vertice inicial:  1263057
Materiais vertice inicial:  1350753
Processando modelo arvore.obj. Vertice final: 1408281


In [3062]:
modelo = load_model_from_file('pato/pato.obj')

print('Processando modelo pato.obj. Vertice inicial:',len(vertices_list))
ini_pato = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo pato.obj. Vertice final:',len(vertices_list))
fim_pato = len(vertices_list)

load_texture_from_file(17,'pato/pato.jpg')

Processando modelo pato.obj. Vertice inicial: 1408281
Processando modelo pato.obj. Vertice final: 1460793


In [3063]:
modelo = load_model_from_file('mesa/mesa.obj')

print('Processando modelo mesa.obj. Vertice inicial:',len(vertices_list))
ini_mesa = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo mesa.obj. Vertice final:',len(vertices_list))
fim_mesa = len(vertices_list)

load_texture_from_file(18,'mesa/mesa.png')

Processando modelo mesa.obj. Vertice inicial: 1460793
Processando modelo mesa.obj. Vertice final: 1469547


In [3064]:
modelo = load_model_from_file('ceu/ceu.obj')

print('Processando modelo ceu.obj. Vertice inicial:',len(vertices_list))
ini_ceu = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo ceu.obj. Vertice final:',len(vertices_list))
fim_ceu = len(vertices_list)

load_texture_from_file(19,'ceu/ceu.png')

Processando modelo ceu.obj. Vertice inicial: 1469547
Processando modelo ceu.obj. Vertice final: 1472427


In [3065]:
modelo = load_model_from_file('carro/carro.obj')

print('Processando modelo carro.obj. Vertice inicial:',len(vertices_list))
ini_carro = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo carro.obj. Vertice final:',len(vertices_list))
fim_carro = len(vertices_list)

load_texture_from_file(20,'carro/carro.jpg')

Processando modelo carro.obj. Vertice inicial: 1472427
Processando modelo carro.obj. Vertice final: 1486629


In [3066]:
modelo = load_model_from_file('cachorro/cachorro.obj')

print('Processando modelo cachorro.obj. Vertice inicial:',len(vertices_list))
ini_cachorro = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo cachorro.obj. Vertice final:',len(vertices_list))
fim_cachorro = len(vertices_list)

load_texture_from_file(21,'cachorro/cachorro.jpg')

Processando modelo cachorro.obj. Vertice inicial: 1486629
Processando modelo cachorro.obj. Vertice final: 1702533


In [3067]:
modelo = load_model_from_file('homem/homem.obj')

print('Processando modelo homem.obj. Vertice inicial:',len(vertices_list))
ini_homem = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo homem.obj. Vertice final:',len(vertices_list))
fim_homem = len(vertices_list)

load_texture_from_file(22,'homem/homem.jpg')

Processando modelo homem.obj. Vertice inicial: 1702533
Processando modelo homem.obj. Vertice final: 1764159


In [3068]:
modelo = load_model_from_file('cama/cama.obj')

print('Processando modelo cama.obj. Vertice inicial:',len(vertices_list))
ini_cama = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo cama.obj. Vertice final:',len(vertices_list))
fim_cama = len(vertices_list)

load_texture_from_file(23,'cama/cama.tga')

Processando modelo cama.obj. Vertice inicial: 1764159
Processando modelo cama.obj. Vertice final: 2487699


In [3069]:
modelo = load_model_from_file('parede/wall.obj')

print('Processando modelo parede.obj. Vertice inicial:',len(vertices_list))
ini_parede = len(vertices_list)
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo parede.obj. Vertice final:',len(vertices_list))
fim_parede = len(vertices_list)

load_texture_from_file(24,'parede/parede.jpg')

Processando modelo parede.obj. Vertice inicial: 2487699
Processando modelo parede.obj. Vertice final: 2487771


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [3070]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)


###  Enviando coordenadas de vértices para a GPU

In [3071]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


###  Enviando coordenadas de textura para a GPU

In [3072]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo


In [3073]:
def desenha_arvore(_tx, _ty, _tz):
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = _tx; ty = _ty; tz = _tz;
    
    # escala
    sx = 3.0; sy = 3.0; sz = 3.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 0)
    
    for i in range(0, len(arvore_text_id)):
        glBindTexture(GL_TEXTURE_2D, arvore_text_id[i])
        glDrawArrays(GL_TRIANGLES, arvore_vertices_text[i], arvore_vertices_text[i+1]-arvore_vertices_text[i])
    

In [3074]:
def desenha_terreno():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = 0.0; ty = -1.01; tz = 0.0;
    
    # escala
    sx = 40.0; sy = 40.0; sz = 40.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 1)
    
    glDrawArrays(GL_TRIANGLES, ini_pedra, fim_pedra-ini_pedra)
    
    glBindTexture(GL_TEXTURE_2D, 4)
    
    glDrawArrays(GL_TRIANGLES, ini_grama, fim_grama-ini_grama)
    

In [3075]:
def desenha_sofa():
    
    # rotacao
    angle = 270.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = 18.5; ty = -0.9; tz = -5.0;
    
    # escala
    sx = 0.6; sy = 0.6; sz = 0.6;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 13)
    
    glDrawArrays(GL_TRIANGLES, ini_sofa, fim_sofa-ini_sofa)

In [3076]:
def desenha_casa():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = -8.0; ty = -1.0; tz = 5.0;
    
    # escala
    sx = 3.0; sy = 3.0; sz = 3.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    for i in range(0, len(text_id)):
        glBindTexture(GL_TEXTURE_2D, text_id[i])
        glDrawArrays(GL_TRIANGLES, vertices_text[i], vertices_text[i+1]-vertices_text[i])
    

In [3077]:
def desenha_tv():
    
    # rotacao
    angle = -270.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = -3.5; ty = 1.1; tz = -4.5;
    
    # escala
    sx = 0.6; sy = 0.6; sz = 0.6;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 14)
    
    glDrawArrays(GL_TRIANGLES, ini_tv, fim_tv-ini_tv)

In [3078]:
def desenha_pato():
    
    # rotacao
    angle = 0.0;
    rx = -5.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = -20; ty = -1.0; tz = 9.0;
    
    # escala
    sx = 0.3; sy = 0.3; sz = 0.3;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 17)
    
    glDrawArrays(GL_TRIANGLES, ini_pato, fim_pato-ini_pato)

In [3079]:
def desenha_mesa():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = -4.0; ty = -1.0; tz = -4.5;
    
    # escala
    sx = 6.0; sy = 6.0; sz = 6.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 18)
    
    glDrawArrays(GL_TRIANGLES, ini_mesa, fim_mesa-ini_mesa)

In [3080]:
def desenha_ceu():
    
    # rotacao
    angle = 90.0;
    rx = 90.0; ry = 1.0; rz = 90.0;
    
    # translacao
    tx = 40.0; ty = -60.0; tz = -100.5;
    
    # escala
    sx = 20.0; sy = 20.0; sz = 20.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 19)
    
    glDrawArrays(GL_TRIANGLES, ini_ceu, fim_ceu-ini_ceu)

In [3081]:
def desenha_carro():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 0.0; rz = 1.0;
    
    # translacao
    tx = -20; ty = -1.0; tz = -5.0;
    
    # escala
    sx = 4.0; sy = 4.0; sz = 4.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    glBindTexture(GL_TEXTURE_2D, 20)
    
    glDrawArrays(GL_TRIANGLES, ini_carro, fim_carro-ini_carro)

In [3082]:
def desenha_cachorro():
    
    # rotacao
    angle = 0.0;
    rx = -5.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = 25; ty = -1.0; tz = 9.0;
    
    # escala
    sx = 0.15; sy = 0.15; sz = 0.15;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 21)
    
    glDrawArrays(GL_TRIANGLES, ini_cachorro, fim_cachorro-ini_cachorro)

In [3083]:
def desenha_homem():

    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 1.0; rz = 0.0;

    # translacao
    tx = -1.0; ty = -1.0; tz = -15.0;

    # escala
    sx = 0.4; sy = 0.4; sz = 0.4;

    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    glBindTexture(GL_TEXTURE_2D, 22)

    glDrawArrays(GL_TRIANGLES, ini_homem, fim_homem-ini_homem)

In [3084]:
def desenha_cama():
    
    # rotacao
    angle = 0.0;
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # translacao
    tx = 14.5; ty = -1.7; tz = -18.0;
    
    # escala
    sx = 0.05; sy = 0.05; sz = 0.05;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 23)
    
    glDrawArrays(GL_TRIANGLES, ini_cama, fim_cama-ini_cama)

In [3085]:
def desenha_parede(tx, ty, tz, angle):
    
    # rotacao
    rx = 0.0; ry = 1.0; rz = 0.0;
    
    # escala
    sx = 3.0; sy = 3.0; sz = 3.0;
    
    mat_model = model(angle, rx, ry, rz, tx, ty, tz, sx, sy, sz)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    glBindTexture(GL_TEXTURE_2D, 24)
    
    glDrawArrays(GL_TRIANGLES, ini_parede, fim_parede-ini_parede)

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [3086]:
cameraPos   = glm.vec3(10.0,  10.0,  10.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);

def higher_than_ground(pressedW, pressedS, pressedA, pressedD):
    auxW = cameraPos + cameraSpeed * cameraFront
    auxS = cameraPos - cameraSpeed * cameraFront
    
    if(pressedW and auxW[1] > -1):
        return True
    elif(pressedS and auxS[1] > -1):
        return True
    else:
        return False

def inside_wall(pressedW, pressedS, pressedA, pressedD):
    auxW = cameraPos + cameraSpeed * cameraFront
    auxS = cameraPos - cameraSpeed * cameraFront
    auxA = cameraPos - glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
    auxD = cameraPos + glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if(pressedW and auxW[2] > -48.25 and auxW[2] < 38.25 and auxW[0] > -48 and auxW[0] < 48 and auxW[1] < 22.6117):
        return True
    elif(pressedS and auxS[2] > -48.25 and auxS[2] < 38.25 and auxS[0] > -48 and auxS[0] < 48 and auxS[1] < 22.6117):
        return True
    elif(pressedA and auxA[2] > -48.25 and auxA[2] < 38.25 and auxA[0] > -48 and auxA[0] < 48 and auxA[1] < 22.6117):
        return True
    elif(pressedD and auxD[2] > -48.25 and auxD[2] < 38.25 and auxD[0] > -48 and auxD[0] < 48 and auxD[1] < 22.6117):
        return True
    
    return False

polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, cameraSpeed, polygonal_mode
    
    print(cameraPos)
    cameraSpeed = 1.5
    if key == 87 and (action==1 or action==2): # tecla W
        if higher_than_ground(True, False, False, False) and inside_wall(True, False, False, False):
            cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        if higher_than_ground(False, True, False, False) and inside_wall(False, True, False, False):
            cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        if inside_wall(False, False, True, False):
            cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        if inside_wall(False, False, False, True):
            cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)


### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [3087]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade

    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform).T # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, nós exibimos a janela!


In [3088]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [3089]:
glEnable(GL_DEPTH_TEST) ### importante para 3D
   

rotacao_inc = 0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    desenha_arvore(30.0, -1.0, 30.0)
    desenha_arvore(-30.0, -1.0, 30.0)   
    desenha_terreno()
    desenha_casa()
    desenha_sofa()
    desenha_tv()
    desenha_pato()
    desenha_mesa()
    desenha_ceu()
    desenha_carro()
    desenha_cachorro()
    desenha_homem()
    desenha_cama()
    desenha_parede(50.0, -1.7, 20.25, 90)
    desenha_parede(50.0, -1.7, -20.25, 90)
    desenha_parede(50.0, -1.7, 60.75, 90)
    desenha_parede(20.25, -1.7, -50.25, 0)
    desenha_parede(-20.25, -1.7, -50.25, 0)
    desenha_parede(-60.75, -1.7, -50.25, 0)
    desenha_parede(-50.0, -1.7, 20.25, 90)
    desenha_parede(-50.0, -1.7, -20.25, 90)
    desenha_parede(-50.0, -1.7, 60.75, 90)
    desenha_parede(20.25, -1.7, 40.25, 0)
    desenha_parede(-20.25, -1.7, 40.25, 0)
    desenha_parede(-60.75, -1.7, 40.25, 0)
    #x = 48 -48
    #z = -48.25 38.25 
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)    

    
    glfw.swap_buffers(window)

glfw.terminate()

vec3(           10,           10,           10 )
vec3(      10.9652,      8.88004,      10.2532 )
vec3(      12.3398,       9.3211,      10.6604 )
vec3(      13.7011,      9.82921,      11.0328 )
vec3(      15.0305,      10.4465,       11.352 )
vec3(      16.3039,      11.1965,      11.6087 )
vec3(      17.4601,      12.1343,      11.7918 )
vec3(      18.4988,      13.2115,      11.8955 )
vec3(      19.5211,       14.305,      11.9922 )
vec3(      20.5433,      15.3984,      12.0888 )
vec3(      21.5656,      16.4919,      12.1854 )
vec3(      22.5879,      17.5853,      12.2821 )
vec3(      23.6101,      18.6788,      12.3787 )
vec3(      24.6324,      19.7723,      12.4753 )
vec3(      25.6388,      20.8817,      12.5545 )
vec3(      26.6353,      22.0017,      12.6067 )
vec3(      26.6353,      22.0017,      12.6067 )
vec3(      26.6353,      22.0017,      12.6067 )
vec3(      26.6353,      22.0017,      12.6067 )
vec3(      26.6353,      22.0017,      12.6067 )
vec3(      26.6353, 

vec3(     -45.6282,      22.5832,     -25.2631 )
vec3(      -45.378,      22.5832,     -26.7421 )
vec3(     -45.1279,      22.5832,     -28.2211 )
vec3(     -44.8777,      22.5832,     -29.7001 )
vec3(     -44.6276,      22.5832,      -31.179 )
vec3(     -44.3774,      22.5832,      -32.658 )
vec3(     -44.1273,      22.5832,      -34.137 )
vec3(     -43.8771,      22.5832,      -35.616 )
vec3(     -43.6269,      22.5832,      -37.095 )
vec3(     -43.3768,      22.5832,      -38.574 )
vec3(     -43.1266,      22.5832,      -40.053 )
vec3(     -42.8765,      22.5832,      -41.532 )
vec3(     -42.6263,      22.5832,      -43.011 )
vec3(     -42.3839,      22.5832,     -44.4913 )
vec3(     -42.1493,      22.5832,     -45.9728 )
vec3(     -41.9146,      22.5832,     -47.4543 )
vec3(     -41.9146,      22.5832,     -47.4543 )
vec3(     -41.9146,      22.5832,     -47.4543 )
vec3(     -41.9146,      22.5832,     -47.4543 )
vec3(     -43.4028,      22.5283,     -47.6344 )
vec3(     -43.4028, 

vec3(      46.7769,      22.5989,     -40.3736 )
vec3(      46.7769,      22.5989,     -40.3736 )
vec3(      46.7769,      22.5989,     -40.3736 )
vec3(      46.7769,      22.5989,     -40.3736 )
vec3(      46.7769,      22.5989,     -40.3736 )
vec3(      46.7769,      22.5989,     -40.3736 )
vec3(      46.7769,      22.5989,     -40.3736 )
vec3(      45.2907,      22.5989,     -40.5772 )
vec3(      45.2907,      22.5989,     -40.5772 )
vec3(      45.2907,      22.5989,     -40.5772 )
vec3(      45.2907,      22.5989,     -40.5772 )
vec3(      45.2907,      22.5989,     -40.5772 )
vec3(      45.2907,      22.5989,     -40.5772 )
vec3(      45.2907,      22.5989,     -40.5772 )
vec3(      45.2907,      22.5989,     -40.5772 )
vec3(      45.2907,      22.5989,     -40.5772 )
vec3(      45.2907,      22.5989,     -40.5772 )
vec3(      43.7954,      22.5989,     -40.6949 )
vec3(      43.7954,      22.5989,     -40.6949 )
vec3(      43.7954,      22.5989,     -40.6949 )
vec3(      43.6624, 

vec3(      47.9212,      18.3062,      38.2211 )
vec3(      47.9212,      18.3062,      38.2211 )
vec3(      47.6324,      18.3062,      36.7491 )
vec3(      47.6324,      18.3062,      36.7491 )
vec3(      46.3415,      18.3298,      37.5126 )
vec3(      46.3415,      18.3298,      37.5126 )
vec3(      45.1372,      18.3298,      36.6183 )
vec3(      45.1372,      18.3298,      36.6183 )
vec3(      43.6439,      18.3298,      36.4771 )
vec3(      42.1449,      18.3298,      36.5321 )
vec3(      40.6449,      18.3298,      36.5399 )
vec3(      39.1454,      18.3298,      36.5007 )
vec3(      37.6475,      18.3298,      36.4222 )
vec3(      36.1504,      18.3298,       36.328 )
vec3(      34.6534,      18.3298,      36.2338 )
vec3(      33.1564,      18.3298,      36.1396 )
vec3(      31.6593,      18.3298,      36.0454 )
vec3(      30.1623,      18.3298,      35.9512 )
vec3(      28.6652,      18.3298,       35.857 )
vec3(      27.1682,      18.3298,      35.7629 )
vec3(      25.6712, 

vec3(     -45.8177,      21.8978,      38.0619 )
vec3(     -45.8177,      21.8978,      38.0619 )
vec3(     -45.8177,      21.8978,      38.0619 )
vec3(     -45.8177,      21.8978,      38.0619 )
vec3(     -45.8177,      21.8978,      38.0619 )
vec3(     -44.4645,      21.4718,      37.5747 )
vec3(     -43.0836,      21.5973,      37.0027 )
vec3(     -41.7379,      21.7619,      36.3608 )
vec3(     -40.3922,      21.9265,      35.7189 )
vec3(     -39.0465,      22.0911,      35.0771 )
vec3(      -37.711,      22.2557,      34.4141 )
vec3(      -36.379,      22.4203,      33.7442 )
vec3(     -35.0471,      22.5849,      33.0743 )
vec3(     -35.0471,      22.5849,      33.0743 )
vec3(     -35.0471,      22.5849,      33.0743 )
vec3(     -35.0471,      22.5849,      33.0743 )
vec3(     -35.0471,      22.5849,      33.0743 )
vec3(     -35.0471,      22.5849,      33.0743 )
vec3(     -35.0471,      22.5849,      33.0743 )
vec3(     -35.0471,      22.5849,      33.0743 )
vec3(     -34.0111, 

vec3(      12.2829,      21.6863,     -41.6117 )
vec3(      10.7836,      21.6863,     -41.6588 )
vec3(      9.28398,      21.6863,     -41.6274 )
vec3(      7.78564,      21.6863,     -41.5567 )
vec3(      6.28967,      21.6863,     -41.4468 )
vec3(       4.8015,      21.6863,     -41.2588 )
vec3(      3.31646,      21.6863,     -41.0475 )
vec3(      1.83493,      21.6863,     -40.8128 )
vec3(     0.360048,      21.6863,     -40.5395 )
vec3(     -1.10209,      21.6863,     -40.2046 )
vec3(     -2.55497,      21.6863,     -39.8316 )
vec3(     -4.00386,      21.6863,     -39.4433 )
vec3(     -5.43522,      21.6863,     -38.9948 )
vec3(     -6.83838,      21.6863,     -38.4646 )
vec3(     -8.22118,      21.6863,     -37.8833 )
vec3(      -9.5915,      21.6863,     -37.2732 )
vec3(     -10.8944,      21.6863,       -36.53 )
vec3(     -12.0299,      21.6863,     -35.5499 )
vec3(     -13.1016,      21.6863,     -34.5004 )
vec3(     -14.1285,      21.6863,     -33.4069 )
vec3(     -15.0846, 